### 样本交易特征统计
- 以账期订单为样本

In [1]:
#读取所有文件
import pandas as pd
import os
import numpy as np
import re
from funcs import *
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')

In [2]:
#统计所有交易特征
def ac_feature(trade,t_now):
    feature = []
    num = trade.shape[0] #订单数
    coo_month = coo_months(trade[u'创建时间'])  #合作时间（月）
    feature.append(num)
    feature.append(coo_month)
    feature.append(recent_month(trade[u'创建时间'],t_now))
    feature.append(recent_last_month(trade[u'创建时间']))
    
    purchase_month = purchase_months(trade[u'创建时间'])
    gap_months = coo_month-purchase_month  #采购断档总月数
    feature.append(gap_months)
    
    tim_st = time_gap(trade[u'创建时间'])
    feature.append(tim_st[0]) #均值
    feature.append(tim_st[1]) #方差
    
    t_num = trade[u'销售数量（吨）'].agg(['sum','max','min','std','mean'])
    t_money = trade[u'销售金额（元）'].agg(['sum','max','min','std','mean'])
    feature.extend(t_num)
    feature.extend(t_money)
    
    feature.append(trade_stab(trade))
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[1]).value_counts().shape[0])
    feature.append(trade[u'货物'].apply(lambda x:x.split(' ')[0]).value_counts().shape[0])
        
    zhangqi = trade[trade[u'是否账期']==u'是']
    if zhangqi.shape[0]>0:
        feature.append(zhangqi.shape[0]*1.0/num)
        feature.append(zhangqi[u'销售数量（吨）'].mean())
        feature.append(zhangqi[u'销售金额（元）'].mean())
        feature.append(coo_months(zhangqi[u'创建时间']))
        feature.append(recent_month(zhangqi[u'创建时间'],t_now))
    else:
        feature.append(0)
        feature.append(0)
        feature.append(0)
        feature.append(0)
        feature.append(np.nan)
    return feature

In [3]:
#统计每个客户账期订单数据
indexs = ['订单总数','合作时间（月）','最近交易（X月之前）','最近连续交易月数','采购断档总月数','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '交易总量','交易量最大值','交易量最小值','交易量方差','交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值',
          '交易稳定性（近三个月交易量变化均值）','供应商数量','采购产品种类数量',
          '账期订单占比','账期订单交易量均值','账期订单交易额均值','账期合作时间（月）','最近一笔账期订单（X月之前）']

def df_sub(record,trade):
    df = pd.DataFrame(index=indexs)    
    tmps = zip(record[u'应收单号'],record[u'起算日期'])
    for t in tmps:
        trade_sub = trade[trade[u'创建时间']<t[1]]
        if trade_sub.shape[0]>0:
            fs = ac_feature(trade_sub,t[1])
        else:
            fs = [np.nan]*len(indexs)
        df[t[0]]=fs
    return df

In [4]:
#所有账期订单数据统计
zq_files = os.listdir('./data/loan_list/')
#所有订单数据
files = VisitDir('./data/client_data/')
files =[f for f in files if '交易记录' in f]

samples = pd.DataFrame()
for f in zq_files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    record = record.iloc[1:,:]  #账期数据
    for f1 in files:
        if f1.split('/')[-2]==f.split('.')[0]:
            trade = pd.read_excel(f1)   #交易数据
            df = df_sub(record,trade)          
            if samples.shape[0]==0:
                samples=df
            else:
                samples = samples.merge(df,left_index=True,right_index=True,how='left')
            continue

samples =samples.T

In [5]:
samples['采购频率（订单总数/合作月数）'] =samples['订单总数']*1.0/samples['合作时间（月）']
samples['采购聚集程度（订单总数/供应商数）'] =samples['订单总数']*1.0/samples['供应商数量']
samples['断档月数占比（断档总月数/合作时间）'] =samples['采购断档总月数']*1.0/samples['合作时间（月）']
samples.to_csv('./data/zq_feature_trade.csv')
samples

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,采购断档总月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,...,供应商数量,采购产品种类数量,账期订单占比,账期订单交易量均值,账期订单交易额均值,账期合作时间（月）,最近一笔账期订单（X月之前）,采购频率（订单总数/合作月数）,采购聚集程度（订单总数/供应商数）,断档月数占比（断档总月数/合作时间）
YS20170228152309,19.0,13.0,1.0,3.0,1.0,20.555556,17.236153,584.75,54.00,20.00,...,5.0,1.0,0.894737,30.455882,188171.029412,12.0,2.0,1.461538,3.800000,0.076923
YS20170103150373,17.0,11.0,3.0,1.0,1.0,19.312500,15.755290,529.75,54.00,20.00,...,5.0,1.0,0.941176,30.609375,188399.218750,10.0,3.0,1.545455,3.400000,0.090909
YS20161117147341,16.0,9.0,1.0,9.0,0.0,16.333333,10.668155,502.75,54.00,20.00,...,5.0,1.0,0.937500,30.850000,189493.166667,8.0,1.0,1.777778,3.200000,0.000000
YS20161115147142,16.0,9.0,1.0,9.0,0.0,16.333333,10.668155,502.75,54.00,20.00,...,5.0,1.0,0.937500,30.850000,189493.166667,8.0,1.0,1.777778,3.200000,0.000000
YS20161025145675,15.0,9.0,0.0,9.0,0.0,16.642857,11.000749,448.75,50.00,20.00,...,5.0,1.0,0.933333,29.196429,174678.392857,8.0,0.0,1.666667,3.000000,0.000000
YS20161019110425,15.0,9.0,0.0,9.0,0.0,16.642857,11.000749,448.75,50.00,20.00,...,5.0,1.0,0.933333,29.196429,174678.392857,8.0,0.0,1.666667,3.000000,0.000000
YS20160926108844,13.0,8.0,1.0,8.0,0.0,17.666667,11.039873,396.75,50.00,20.00,...,5.0,1.0,0.923077,29.729167,172504.791667,7.0,1.0,1.625000,2.600000,0.000000
YS20160926108834,13.0,8.0,1.0,8.0,0.0,17.666667,11.039873,396.75,50.00,20.00,...,5.0,1.0,0.923077,29.729167,172504.791667,7.0,1.0,1.625000,2.600000,0.000000
YS20160824106578,12.0,7.0,0.0,7.0,0.0,16.090909,10.064338,346.75,50.00,20.00,...,5.0,1.0,0.916667,27.886364,156459.772727,6.0,0.0,1.714286,2.400000,0.000000
YS20160811171089,11.0,7.0,0.0,7.0,0.0,16.700000,10.392839,315.75,50.00,20.00,...,5.0,1.0,0.909091,27.575000,153226.750000,6.0,0.0,1.571429,2.200000,0.000000


In [6]:
samples[samples['交易稳定性（近三个月交易量变化均值）']==253.25]

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,采购断档总月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,...,供应商数量,采购产品种类数量,账期订单占比,账期订单交易量均值,账期订单交易额均值,账期合作时间（月）,最近一笔账期订单（X月之前）,采购频率（订单总数/合作月数）,采购聚集程度（订单总数/供应商数）,断档月数占比（断档总月数/合作时间）
YS2016051707046,2.0,1.0,2.0,1.0,0.0,12.0,0.0,255.25,254.25,1.0,...,2.0,1.0,0.0,0.0,0.0,0.0,NaN,2.0,1.0,0.0


### 逾期样本订单

In [14]:
#所有逾期记录
files = os.listdir('./data/loan_list/')
loan_all = pd.DataFrame()
for f in files:
    record = pd.read_excel(os.path.join('./data/loan_list/',f))
    loan_all = loan_all.append(record.iloc[1:,:],ignore_index=True)
loan_all['zq_day'] = loan_all[u'账期'].apply(lambda x:re.findall('\d+',x)[0])
loan_all['yq_day'] = loan_all[u'逾期'].apply(lambda x:re.findall('^(-\d+|\d+)',x)[0])
loan_all['zq_day'] = loan_all['zq_day'].astype(int)
loan_all['yq_day'] = loan_all['yq_day'].apply(lambda x:int(x))
loan_all.to_csv('./data/loan_all.csv',index=None)
loan_all=loan_all.drop_duplicates()
loan_all.shape

(1430, 15)

In [15]:
loan_all['4天以内占比']=0
loan_all['5-10天占比']=0
loan_all['11天以上占比']=0
loan_all[u'逾期占比']=0
loan_all[u'提前还款占比']=0
loan_all.shape

(1430, 20)

In [16]:
def yq_sts(list):
    yq_4=0
    yq_10=0
    yq_11=0
    zq_num = len(list)
    for i in list:
        if i>0 and i<=4:
            yq_4+=1
        elif i>4 and i<=10:
            yq_10+=1
        elif i>10:
            yq_11+=1
            
    return yq_4*1.0/zq_num,yq_10*1.0/zq_num,yq_11*1.0/zq_num

In [17]:
#统计
yq_ratio =[]
tq_ratio =[]
yq_day_ratio =[]
loan_all[u'起算日期'] =pd.to_datetime(loan_all[u'起算日期'])
for i in xrange(loan_all.shape[0]):
    name = loan_all.iat[i,1]
    time = loan_all.iat[i,10]
    loan_sub = loan_all[(loan_all[u'客户'] == name) & (loan_all[u'起算日期']<time)]
    if loan_sub.shape[0]>0:
        tmps = yq_sts(loan_sub['yq_day'])
        yq_day_ratio.append(tmps)
        yq_ratio.append(loan_sub[loan_sub['yq_day']>0].shape[0]*1.0 / loan_sub.shape[0])
        tq_ratio.append( loan_sub[loan_sub['yq_day']<0].shape[0]*1.0 / loan_sub.shape[0])
    else:
        yq_day_ratio.append((0,0,0))
        yq_ratio.append(0)
        tq_ratio.append(0)

loan_all['4天以内占比']=[x[0] for x in yq_day_ratio]
loan_all['5-10天占比']=[x[1] for x in yq_day_ratio]
loan_all['11天以上占比']=[x[2] for x in yq_day_ratio]
loan_all[u'逾期占比']=yq_ratio
loan_all[u'提前还款占比']=tq_ratio
loan_all.to_csv('./data/loan_all.csv')
loan_all.head()

,业务类型,客户,已收金额,应收,应收余额,应收单号,应收总额,应收日期,收款状态,账期,起算日期,逾期,销售合同号,zq_day,yq_day,4天以内占比,5-10天占比,11天以上占比,逾期占比,提前还款占比
0,账期业务,佛山市南海区松岗明珠塑料有限公司,184520.0,NaN,0.0,YS20170228152309,184520.0,2017-03-30,已收款,30天,2017-02-28,11天,ZSL20170227LL-C003WJ,30,11,0.157895,0.0,0.0,0.157895,0.263158
1,账期业务,佛山市南海区松岗明珠塑料有限公司,171990.0,NaN,0.0,YS20170103150373,171990.0,2017-01-13,已收款,10天,2017-01-03,0天,ZSL20161228LL-C041WJ,10,0,0.166667,0.0,0.0,0.166667,0.277778
2,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161117147341,198450.0,2016-12-02,已收款,15天,2016-11-17,-2天,ZSL20161025LL-C009WJ,15,-2,0.176471,0.0,0.0,0.176471,0.235294
3,账期业务,佛山市南海区松岗明珠塑料有限公司,198450.0,NaN,0.0,YS20161115147142,198450.0,2016-11-29,已收款,15天,2016-11-14,0天,ZSL20161025LL-C009WJ,15,0,0.187500,0.0,0.0,0.187500,0.250000
4,账期业务,佛山市南海区松岗明珠塑料有限公司,186940.0,NaN,0.0,YS20161025145675,186940.0,2016-11-09,已收款,15天,2016-10-25,0天,ZSL20161011LL-C038WJ,15,0,0.200000,0.0,0.0,0.200000,0.266667


In [21]:
loan_all[loan_all['应收单号']=='YS2016051707046']

,Unnamed: 0,业务类型,客户,已收金额,应收,应收余额,应收单号,应收总额,应收日期,收款状态,...,起算日期,逾期,销售合同号,zq_day,yq_day,4天以内占比,5-10天占比,11天以上占比,逾期占比,提前还款占比
671,671,账期业务,开平市源成达塑胶制品有限公司,50700.0,NaN,0.0,YS2016051707046,50700.0,2016-06-01,已收款,...,2016-05-17,-1天,ZSL20160517WXB-C217J,15,-1,0.0,0.0,0.0,0.0,0.0


### 合并样本特征

In [7]:
#账期订单特征
samples = pd.read_csv('./data/zq_feature_trade.csv',index_col=0)
samples.index = [x.encode('utf-8') for x in samples.index]
loan_all = pd.read_csv('./data/loan_all.csv')
tmp_loan = loan_all[['应收单号','4天以内占比', '5-10天占比', '11天以上占比','逾期占比','提前还款占比','yq_day','zq_day','客户','收款状态']]
samples_all = samples.merge(tmp_loan,left_index=True,right_on='应收单号',how='left')
samples_all = samples_all.set_index('应收单号')
samples_all.shape

(1430, 37)

In [8]:
#客户公司基础特征
feature_card = pd.read_csv('./data/features_card.csv',index_col=0)
samples_all = samples_all.merge(feature_card,left_on='客户',right_index=True,how='left')
samples_all =samples_all.drop(['客户'],axis=1)
samples_all.head()

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,采购断档总月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,...,流动比率,净资产,货币资金,存货周转天数,应收账款周转天数,主营业务利润率,净资产增长率,销售增长率,当地经济环境,撮合单数
应收单号,,,,,,,,,,,,,,,,,,,,,
YS20170228152309,19.0,13.0,1.0,3.0,1.0,20.555556,17.236153,584.75,54.0,20.0,...,132.0,226.0,64.0,13.0,43.0,8.0,-7.0,1.0,8.3,0.0
YS20170103150373,17.0,11.0,3.0,1.0,1.0,19.312500,15.755290,529.75,54.0,20.0,...,132.0,226.0,64.0,13.0,43.0,8.0,-7.0,1.0,8.3,0.0
YS20161117147341,16.0,9.0,1.0,9.0,0.0,16.333333,10.668155,502.75,54.0,20.0,...,132.0,226.0,64.0,13.0,43.0,8.0,-7.0,1.0,8.3,0.0
YS20161115147142,16.0,9.0,1.0,9.0,0.0,16.333333,10.668155,502.75,54.0,20.0,...,132.0,226.0,64.0,13.0,43.0,8.0,-7.0,1.0,8.3,0.0
YS20161025145675,15.0,9.0,0.0,9.0,0.0,16.642857,11.000749,448.75,50.0,20.0,...,132.0,226.0,64.0,13.0,43.0,8.0,-7.0,1.0,8.3,0.0


### 按账期分产品
- 账期<=7天，逾期大于5天为坏客户
- 账期8-15天，逾期大于4天为坏客户
- 账期>15天，逾期大于7天为坏客户

In [9]:
samples_all = samples_all[samples_all['收款状态']=='已收款']
zq_7 = samples_all[samples_all.zq_day<=7].drop(['zq_day','收款状态'],axis=1)
zq_15 = samples_all[(samples_all.zq_day>7) & (samples_all.zq_day<=15)].drop(['zq_day','收款状态'],axis=1)
zq_30 = samples_all[samples_all.zq_day>15].drop(['zq_day','收款状态'],axis=1)

In [10]:
zq_7['label'] = zq_7['yq_day'].apply(lambda x:1 if x>4 else 0)
zq_7 =zq_7.drop(['yq_day'],axis=1)
zq_15['label'] = zq_15['yq_day'].apply(lambda x:1 if x>3 else 0)
zq_15 =zq_15.drop(['yq_day'],axis=1)
zq_30['label'] = zq_30['yq_day'].apply(lambda x:1 if x>6 else 0)
zq_30 =zq_30.drop(['yq_day'],axis=1)

In [33]:
zq_7.to_csv('./data/samples_7.csv')
zq_15.to_csv('./data/samples_15.csv')
zq_30.to_csv('./data/samples_30.csv')
zq_all = zq_7.append(zq_15)
zq_all =zq_all.append(zq_30)
zq_all.to_csv('./data/samples_all.csv')

In [34]:
zq_all.shape

(1391, 63)

In [73]:
#归一化
# t_cols =['订单总数','合作时间（月）','最近交易（X月之前）','最近连续交易月数','订单时间间隔均值（天）','交易总量','交易量最大值',
#          '交易量最小值','交易量方差','交易量均值','交易额总量','交易额最大值','交易额最小值','交易额方差','交易额均值','供应商数量',
#          '采购产品种类数量','账期订单交易量均值','账期订单交易额均值','账期合作时间（月）','最近一笔账期订单（X月之前）','注册时间',
#          '实收资本','员工人数','厂房面积','资产负债率','流动比率','净资产','货币资金','存货周转天数','应收账款周转天数','主营业务利润率',
#          '净资产增长率','销售增长率','当地经济环境','撮合单数','交易稳定性（近三个月交易量变化均值）']
# for col in t_cols:
#     zq_7[col] =normalize(zq_7[col])
#     zq_15[col] =normalize(zq_15[col])
#     zq_30[col] =normalize(zq_30[col])
# zq_7.to_csv('./data/samples_7_nor.csv')
# zq_15.to_csv('./data/samples_15_nor.csv')
# zq_30.to_csv('./data/samples_30_nor.csv')
# zq_all_nor = zq_7.append(zq_15)
# zq_all_nor =zq_all_nor.append(zq_30)
# zq_all_nor.to_csv('./data/samples_all_nor.csv')

### 统计量，计算IV值,特征相关性

In [15]:
#std,min,max
features_st = samples_all.describe().T
features_st['null_ratio']=pd.DataFrame.from_dict([null_ratio(samples_all)]).T
features_st

,count,mean,std,min,25%,50%,75%,max,null_ratio
订单总数,1389.0,1.623902e+01,1.232207e+01,1.000000,8.000000e+00,1.300000e+01,2.100000e+01,7.300000e+01,0.001438
合作时间（月）,1389.0,7.912887e+00,4.416570e+00,1.000000,5.000000e+00,7.000000e+00,1.100000e+01,2.200000e+01,0.001438
最近交易（X月之前）,1366.0,8.008785e-01,1.113960e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,0.017973
最近连续交易月数,1389.0,4.539237e+00,3.481229e+00,1.000000,2.000000e+00,4.000000e+00,6.000000e+00,2.200000e+01,0.001438
采购断档总月数,1389.0,1.117351e+00,1.915332e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.200000e+01,0.001438
订单时间间隔均值（天）,1362.0,2.074079e+01,2.323819e+01,2.500000,1.231579e+01,1.655556e+01,2.198214e+01,3.250000e+02,0.020848
订单时间间隔方差（天）,1362.0,1.502101e+01,1.454600e+01,0.000000,8.020806e+00,1.193099e+01,1.716021e+01,1.359130e+02,0.020848
交易总量,1389.0,8.563054e+02,9.454483e+02,2.000000,2.489800e+02,4.960000e+02,1.084300e+03,4.637350e+03,0.001438
交易量最大值,1389.0,1.086928e+02,9.502928e+01,2.000000,4.000000e+01,7.800000e+01,1.550000e+02,4.030000e+02,0.001438
交易量最小值,1389.0,1.542819e+01,1.230151e+01,0.050000,2.000000e+00,1.600000e+01,2.650000e+01,1.130000e+02,0.001438


In [16]:
#ivs
features_st['ivs'] = pd.DataFrame.from_dict([calc_iv(zq_all,box_num=3)]).T.sort_values(by=0,ascending=False)
features_st

,count,mean,std,min,25%,50%,75%,max,null_ratio,ivs
订单总数,1389.0,1.623902e+01,1.232207e+01,1.000000,8.000000e+00,1.300000e+01,2.100000e+01,7.300000e+01,0.001438,0.019077
合作时间（月）,1389.0,7.912887e+00,4.416570e+00,1.000000,5.000000e+00,7.000000e+00,1.100000e+01,2.200000e+01,0.001438,0.060901
最近交易（X月之前）,1366.0,8.008785e-01,1.113960e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,0.017973,0.008802
最近连续交易月数,1389.0,4.539237e+00,3.481229e+00,1.000000,2.000000e+00,4.000000e+00,6.000000e+00,2.200000e+01,0.001438,0.023032
采购断档总月数,1389.0,1.117351e+00,1.915332e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.200000e+01,0.001438,0.015771
订单时间间隔均值（天）,1362.0,2.074079e+01,2.323819e+01,2.500000,1.231579e+01,1.655556e+01,2.198214e+01,3.250000e+02,0.020848,0.065196
订单时间间隔方差（天）,1362.0,1.502101e+01,1.454600e+01,0.000000,8.020806e+00,1.193099e+01,1.716021e+01,1.359130e+02,0.020848,0.062602
交易总量,1389.0,8.563054e+02,9.454483e+02,2.000000,2.489800e+02,4.960000e+02,1.084300e+03,4.637350e+03,0.001438,0.055115
交易量最大值,1389.0,1.086928e+02,9.502928e+01,2.000000,4.000000e+01,7.800000e+01,1.550000e+02,4.030000e+02,0.001438,0.046432
交易量最小值,1389.0,1.542819e+01,1.230151e+01,0.050000,2.000000e+00,1.600000e+01,2.650000e+01,1.130000e+02,0.001438,0.126815


In [17]:
#pearson
label_corr = zq_all.corr()['label']
yq_corr = samples_all.corr()['yq_day']
t = features_st.reset_index().merge(yq_corr.reset_index(),on='index',how='left')
t = t.merge(label_corr.reset_index(),on='index',how='left')
t.set_index('index',inplace=True)
# t.to_csv('./data/feature_st.csv')

#调整顺序
indexs = ['实收资本','员工人数','厂房面积','场地归属','注册时间','商标专利','注册地址','注册资本变更','分支机构','法人变更',
          '流动比率','资产负债率','存货周转天数','货币资金','净资产','主营业务利润率','应收账款周转天数','净资产增长率','销售增长率',
          '下游客户情况','当地经济环境','企业征信','涉诉信息','有无贷款','提前还款占比','逾期占比','账期订单交易量均值','交易量方差',
          '交易量最小值','交易额最小值','交易量均值','采购频率（订单总数/合作月数）','供应商数量','交易额均值','账期订单交易额均值',
          '交易额方差','交易稳定性（近三个月交易量变化均值）','交易额最大值','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '合作时间（月）','账期订单占比','交易额总量','交易总量','交易量最大值','最近连续交易月数','账期合作时间（月）',
          '采购聚集程度（订单总数/供应商数）','订单总数','采购断档总月数','采购产品种类数量','断档月数占比（断档总月数/合作时间）',
          '最近一笔账期订单（X月之前）','最近交易（X月之前）','4天以内占比','5-10天占比','11天以上占比']

t.loc[indexs,:].to_csv('./data/feature_st.csv')

,count,mean,std,min,25%,50%,75%,max,null_ratio,ivs,yq_day,label
index,,,,,,,,,,,,
订单总数,1389.0,1.623902e+01,1.232207e+01,1.000000,8.000000e+00,1.300000e+01,2.100000e+01,7.300000e+01,0.001438,0.019077,0.007402,0.051066
合作时间（月）,1389.0,7.912887e+00,4.416570e+00,1.000000,5.000000e+00,7.000000e+00,1.100000e+01,2.200000e+01,0.001438,0.060901,-0.057375,-0.037730
最近交易（X月之前）,1366.0,8.008785e-01,1.113960e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.100000e+01,0.017973,0.008802,-0.145019,-0.023804
最近连续交易月数,1389.0,4.539237e+00,3.481229e+00,1.000000,2.000000e+00,4.000000e+00,6.000000e+00,2.200000e+01,0.001438,0.023032,0.043590,0.023511
采购断档总月数,1389.0,1.117351e+00,1.915332e+00,0.000000,0.000000e+00,1.000000e+00,1.000000e+00,1.200000e+01,0.001438,0.015771,-0.052598,-0.050600
订单时间间隔均值（天）,1362.0,2.074079e+01,2.323819e+01,2.500000,1.231579e+01,1.655556e+01,2.198214e+01,3.250000e+02,0.020848,0.065196,-0.080350,-0.071595
订单时间间隔方差（天）,1362.0,1.502101e+01,1.454600e+01,0.000000,8.020806e+00,1.193099e+01,1.716021e+01,1.359130e+02,0.020848,0.062602,-0.040761,-0.056829
交易总量,1389.0,8.563054e+02,9.454483e+02,2.000000,2.489800e+02,4.960000e+02,1.084300e+03,4.637350e+03,0.001438,0.055115,0.017647,0.109938
交易量最大值,1389.0,1.086928e+02,9.502928e+01,2.000000,4.000000e+01,7.800000e+01,1.550000e+02,4.030000e+02,0.001438,0.046432,-0.008373,0.094527


In [18]:
#调整顺序
indexs = ['实收资本','员工人数','厂房面积','场地归属','注册时间','商标专利','注册地址','注册资本变更','分支机构','法人变更',
          '流动比率','资产负债率','存货周转天数','货币资金','净资产','主营业务利润率','应收账款周转天数','净资产增长率','销售增长率',
          '下游客户情况','当地经济环境','企业征信','涉诉信息','有无贷款','提前还款占比','逾期占比','账期订单交易量均值','交易量方差',
          '交易量最小值','交易额最小值','交易量均值','采购频率（订单总数/合作月数）','供应商数量','交易额均值','账期订单交易额均值',
          '交易额方差','交易稳定性（近三个月交易量变化均值）','交易额最大值','订单时间间隔均值（天）','订单时间间隔方差（天）',
          '合作时间（月）','账期订单占比','交易额总量','交易总量','交易量最大值','最近连续交易月数','账期合作时间（月）',
          '采购聚集程度（订单总数/供应商数）','订单总数','采购断档总月数','采购产品种类数量','断档月数占比（断档总月数/合作时间）',
          '最近一笔账期订单（X月之前）','最近交易（X月之前）','4天以内占比','5-10天占比','11天以上占比']

t.loc[indexs,:].to_csv('./data/feature_st.csv')

### 预处理

In [37]:
#缺失值填充
zero_fill =['场地租赁','场地自有','商标专利','注册地址','注册资本变更','分支机构','法人变更','征信关注','涉诉信息','征信不良','有无贷款','征信欠息','征信正常']
for col in zero_fill:
    zq_all[col].fillna(0,inplace=True)
#均值填充
null_col = pd.DataFrame.from_dict([null_ratio(zq_all)]).T
cols = null_col[null_col[0]>0].index
for col in cols:
    zq_all[col].fillna(zq_all[col].mean(),inplace=True)
zq_all.to_csv('./data/samples_all_fillna.csv')
pd.DataFrame.from_dict([null_ratio(zq_all)]).T

,0
11天以上占比,0.0
4天以内占比,0.0
5-10天占比,0.0
label,0.0
下游客户情况,0.0
主营业务利润率,0.0
交易总量,0.0
交易稳定性（近三个月交易量变化均值）,0.0
交易量均值,0.0
交易量方差,0.0


In [39]:
#归一化
for col in cols:
    if col not in ['下游客户情况','断档月数占比（断档总月数/合作时间）','账期订单占比']:
        zq_all[col] = normalize(zq_all[col])
zq_all.to_csv('./data/samples_all_fillna_nor.csv')
zq_all

,订单总数,合作时间（月）,最近交易（X月之前）,最近连续交易月数,采购断档总月数,订单时间间隔均值（天）,订单时间间隔方差（天）,交易总量,交易量最大值,交易量最小值,...,净资产,货币资金,存货周转天数,应收账款周转天数,主营业务利润率,净资产增长率,销售增长率,当地经济环境,撮合单数,label
应收单号,,,,,,,,,,,,,,,,,,,,,
YS2016080317738,0.125000,0.285714,0.090909,0.285714,0.000000,0.047373,0.076620,0.060999,0.119701,0.176627,...,0.042011,0.028881,0.016477,0.146758,0.241379,0.006938,0.034799,0.875000,0.000000,0
YS2016071915487,0.111111,0.238095,0.090909,0.238095,0.000000,0.048837,0.081149,0.056684,0.119701,0.176627,...,0.042011,0.028881,0.016477,0.146758,0.241379,0.006938,0.034799,0.875000,0.000000,0
YS2016070113517,0.097222,0.190476,0.090909,0.190476,0.000000,0.045404,0.084044,0.052369,0.119701,0.176627,...,0.042011,0.028881,0.016477,0.146758,0.241379,0.006938,0.034799,0.875000,0.000000,0
YS2016050505611,0.069444,0.142857,0.090909,0.142857,0.000000,0.036279,0.045176,0.040180,0.119701,0.176627,...,0.042011,0.028881,0.016477,0.146758,0.241379,0.006938,0.034799,0.875000,0.000000,0
YS20160913107905,0.138889,0.428571,0.181818,0.000000,0.166667,0.072868,0.155152,0.111588,0.144638,0.247455,...,0.349199,0.427347,0.103929,0.037543,0.206897,0.019889,0.043956,0.850000,0.046296,1
YS20160911107730,0.138889,0.428571,0.181818,0.000000,0.166667,0.072868,0.155152,0.111588,0.144638,0.247455,...,0.349199,0.427347,0.103929,0.037543,0.206897,0.019889,0.043956,0.850000,0.046296,1
YS2016072516482,0.125000,0.333333,0.090909,0.095238,0.083333,0.064944,0.151999,0.098644,0.144638,0.247455,...,0.349199,0.427347,0.103929,0.037543,0.206897,0.019889,0.043956,0.850000,0.046296,0
YS2016072516481,0.125000,0.333333,0.090909,0.095238,0.083333,0.064944,0.151999,0.098644,0.144638,0.247455,...,0.349199,0.427347,0.103929,0.037543,0.206897,0.019889,0.043956,0.850000,0.046296,0
YS2016070714094,0.111111,0.285714,0.090909,0.047619,0.083333,0.065504,0.162438,0.086455,0.144638,0.247455,...,0.349199,0.427347,0.103929,0.037543,0.206897,0.019889,0.043956,0.850000,0.046296,0


In [50]:
ivs_7 = pd.DataFrame.from_dict([calc_iv(zq_7,box_num=3)]).T
ivs_7.sort_values(by=0,ascending=False).to_csv('./data/features_ivs_zq_7.csv')
ivs_7.sort_values(by=0,ascending=False)

,0
提前还款占比,0.726691
下游客户情况,0.708709
征信关注,0.644166
员工人数,0.618418
注册时间,0.554756
逾期占比,0.545867
交易量均值,0.534601
交易量方差,0.525523
实收资本,0.510972
交易额方差,0.499343


In [51]:
ivs_15 = pd.DataFrame.from_dict([calc_iv(zq_15,box_num=3)]).T
ivs_15.sort_values(by=0,ascending=False).to_csv('./data/features_ivs_zq_15.csv')
ivs_15.sort_values(by=0,ascending=False)

,0
注册时间,0.604317
实收资本,0.576618
应收账款周转天数,0.566900
资产负债率,0.550702
提前还款占比,0.526196
主营业务利润率,0.457387
逾期占比,0.339988
账期订单占比,0.334333
存货周转天数,0.313965
流动比率,0.310079


In [52]:
ivs_30 = pd.DataFrame.from_dict([calc_iv(zq_30,box_num=2)]).T
ivs_30.sort_values(by=0,ascending=False).to_csv('./data/features_ivs_zq_30.csv')
ivs_30.sort_values(by=0,ascending=False)

,0
下游客户情况,0.604152
主营业务利润率,0.598010
货币资金,0.583808
交易额最小值,0.516954
实收资本,0.497081
注册资本变更,0.426077
应收账款周转天数,0.422430
提前还款占比,0.408068
交易量最小值,0.364195
销售增长率,0.347467


In [15]:
#重复订单
tmp = samples_all.groupby(by=[u'应收单号']).count()['订单总数']
l = tmp[tmp>1].index
loan_all[loan_all[u'应收单号'].isin(l)].to_csv('./data/tmp.csv')
